In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("EV-Streaming")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "4")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/17 18:04:10 WARN Utils: Your hostname, RoeisLaptop, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/17 18:04:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/mnt/c/Users/roeiu/OneDrive/Documents/Big%20Data%20-%20Course/pyspark/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/roei/.ivy2.5.2/cache
The jars for the packages stored in: /home/roei/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3dae7271-9676-424c-9680-24fa97bb27f5;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.1.1 in central
	found org.apache

In [2]:
from pyspark.sql.functions import col

kafka_df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "electric_cars")
    .option("startingOffsets", "earliest")
    .load()
)

values_df = kafka_df.select(col("value").cast("string").alias("json_str"))
values_df.printSchema()


root
 |-- json_str: string (nullable = true)



In [3]:
q_raw = (
    values_df.writeStream
    .format("console")
    .option("truncate", "false")
    .start()
)


26/01/17 18:04:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7d3b418d-4e6a-4a50-98d9-0b33a4722250. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
26/01/17 18:04:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [4]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StringType

schema = StructType() \
    .add("City", StringType()) \
    .add("Model Year", StringType())

parsed_df = values_df.select(
    from_json(col("json_str"), schema).alias("data")
).select("data.*")


In [5]:
def top_cities_2023(batch_df, batch_id):
    (
        batch_df
        .filter(col("Model Year") == "2023")
        .groupBy("City")
        .count()
        .orderBy(col("count").desc())
        .limit(3)
        .show(truncate=False)
    )

query = (
    parsed_df.writeStream
    .foreachBatch(top_cities_2023)
    .outputMode("update")
    .start()
)


26/01/17 18:04:52 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ffd8aea3-2f3c-43de-b258-01a7d3b3d85e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
26/01/17 18:04:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|json_str                                                                                                                                                                                                                                                                                                    